In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from linearmodels.iv import IV2SLS
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
from statsmodels.iolib.summary2 import summary_col


In [73]:
df = pd.read_csv("welfare.csv")


In [74]:
df.dropna(inplace = True)

In [75]:
df.isnull().sum()

imm                    0
hsgrad                 0
agelt25                0
age35p                 0
treatment              0
working_at_baseline    0
anykidsu6              0
nevermarried           0
ft15                   0
ft20                   0
ft24                   0
ft48                   0
welfare15              0
welfare20              0
welfare24              0
welfare48              0
dtype: int64

#### Estimating the first stage Model

##### A) Estimation of the first stage model:

In [34]:
def Compute_First_Stage(dta, ft):
    Y = dta[ft]
    X = dta['treatment']
    X = sm.add_constant(X)
    
    reg = sm.OLS(Y,X).fit()
    
    return reg

#Estimate the first stage models for the probability of 
#working FT

ft_15 = Compute_First_Stage(df,'ft15')
ft_20 = Compute_First_Stage(df,'ft20')
ft_24 = Compute_First_Stage(df,'ft24')
ft_48 = Compute_First_Stage(df,'ft48')

In [35]:
def sum_table(results, models, order, name):
    info_dict={'R-squared' : lambda x: f"{x.rsquared:.2f}",
           'No. observations' : lambda x: f"{int(x.nobs):d}"}


    results_table = summary_col(results= results,
                                float_format='%0.4f',
                                stars = True,
                                model_names= models,
                                info_dict=info_dict,
                                regressor_order = order)
    results_table.add_title(name)
    return results_table


In [36]:
First_stage_summary = \
sum_table([ft_15,ft_20,ft_24, ft_48], \
          ['Ft15','Ft20','Ft24','Ft48'], \
         ['constant','treatment'],\
          'Table 1 - First Stage models for the probability of working Ft ')

print(First_stage_summary)

Table 1 - First Stage models for the probability of working Ft 
                    Ft15      Ft20      Ft24      Ft48  
--------------------------------------------------------
treatment        0.1427*** 0.1144*** 0.1075*** 0.0506***
                 (0.0119)  (0.0119)  (0.0118)  (0.0126) 
const            0.1514*** 0.1628*** 0.1612*** 0.2337***
                 (0.0085)  (0.0085)  (0.0084)  (0.0090) 
R-squared        0.03      0.02      0.02      0.00     
No. observations 4796      4796      4796      4796     
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


We can see fromt the first stage regression that: $\gamma_0$

In [37]:
summary = np.vstack([[ft_15.params], [ft_20.params],[ft_24.params], [ft_48.params]])
summary = summary.round(4)

col = ['π0', 'π1']
row = ['Month 15', "Month 20",'Month 24', 'Month 48']
summary_first_stage1 = SimpleTable(summary,col, row, txt_fmt=default_txt_fmt)
print(summary_first_stage1)




           π0     π1  
----------------------
Month 15 0.1514 0.1427
Month 20 0.1628 0.1144
Month 24 0.1612 0.1075
Month 48 0.2337 0.0506
----------------------


In [38]:
def Compute_reduced(dta, welfare):
    Y = dta[welfare]
    X = dta['treatment']
    X = sm.add_constant(X)
    
    reg = sm.OLS(Y,X).fit()
    
    return reg


In [39]:
wf_15 = Compute_reduced(df, 'welfare15')
wf_20 = Compute_reduced(df, 'welfare20')
wf_24 = Compute_reduced(df, 'welfare24')
wf_48 = Compute_reduced(df, 'welfare48')

In [40]:
summary_reduced = \
sum_table([wf_15,wf_20,wf_24, wf_48], \
          ['Welfare 15','Welfare 20','Welfare 24','Welfare 48'], \
         ['constant','treatment'],\
          'Table 2 - Reduced form for the probability of being on Welfare')

print(summary_reduced)



Table 2 - Reduced form for the probability of being on Welfare
                 Welfare 15 Welfare 20 Welfare 24 Welfare 48
------------------------------------------------------------
treatment        -0.1480*** -0.1221*** -0.1075*** -0.0420***
                 (0.0126)   (0.0130)   (0.0133)   (0.0143)  
const            0.8096***  0.7697***  0.7388***  0.5916*** 
                 (0.0090)   (0.0093)   (0.0095)   (0.0102)  
R-squared        0.03       0.02       0.01       0.00      
No. observations 4796       4796       4796       4796      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [41]:
summary = np.vstack([[wf_15.params], [wf_20.params],[wf_24.params], [wf_48.params]])
summary = summary.round(4)

col = ['γ0', 'γ1']
row = ['Month 15', "Month 20",'Month 24', 'Month 48']
summary_first_stage1 = SimpleTable(summary,col, row, txt_fmt=default_txt_fmt)
print(summary_first_stage1)


           γ0      γ1  
-----------------------
Month 15 0.8096  -0.148
Month 20 0.7697 -0.1221
Month 24 0.7388 -0.1075
Month 48 0.5916  -0.042
-----------------------


In [42]:
mod15 = IV2SLS.from_formula('welfare15 ~ 1 + [ft15 ~ treatment]', df).fit()

mod20 = IV2SLS.from_formula('welfare20 ~ 1 + [ft20 ~ treatment]', df).fit()

mod24 = IV2SLS.from_formula('welfare24 ~ 1 + [ft24 ~ treatment]', df).fit()

mod48 = IV2SLS.from_formula('welfare48 ~ 1 + [ft48 ~ treatment]', df).fit()

In [43]:
summary2 = np.vstack([[mod15.params], [mod20.params],[mod24.params], [mod48.params]])
summary2 = summary2.round(4)

col = ['Constant', 'Treatment']
row = ['Month 15', "Month 20",'Month 24', 'Month 48']

summary_second_stage3 = SimpleTable(summary2,col, row, txt_fmt=default_txt_fmt)
print(summary_second_stage3)


         Constant Treatment
---------------------------
Month 15  0.9666    -1.037 
Month 20  0.9434   -1.0667 
Month 24  0.8999     -1.0  
Month 48  0.7855   -0.8299 
---------------------------


| Months | $\beta^IV$|  $\frac{\gamma_1}{\pi_1}$ |
| --- | --- | --- |
| Month 15 | -1.037 | $\frac{-0.148}{0.1427}$ = -1.03714 |
| Month 20 | -1.0667 |$\frac{-0.1221}{0.1144}$ = -1.0673 |
| Month 24 | -1.0  | $\frac{-0.1075}{0.1075}$ = -1.000 |
| Month 48 | -0.8299 |$\frac{-0.042}{0.0506}$ = -0.8300 |

#### Problem 2

In [110]:
############# First Stage ###############
X = df[['treatment', 'imm', 'hsgrad', 'agelt25', 'working_at_baseline',
       'anykidsu6','nevermarried']]
X = sm.add_constant(X)
Y = df['ft15']
reg = sm.OLS(Y,X).fit()

############# Reduced Form###############
X1 = df[['treatment', 'imm', 'hsgrad', 'agelt25', 'working_at_baseline',
       'anykidsu6','nevermarried']]
X1 = sm.add_constant(X)

Y1 = df['welfare15']
reg_rf1 = sm.OLS(Y,X).fit()

############# OLS Casual Model###############
X2 = df[['imm','ft15','hsgrad','agelt25','age35p','working_at_baseline','anykidsu6','nevermarried']]
X2 = sm.add_constant(X2)

Y2 = df['welfare15']

reg_rf2 = sm.OLS(Y2,X2).fit()

############# IV Casual Model###############
X3 = df[['imm','hsgrad','agelt25','age35p','working_at_baseline','anykidsu6','nevermarried']]

mod15 = IV2SLS.from_formula('welfare15 ~ [ft15 ~ treatment] + imm + hsgrad + agelt25 + working_at_baseline + anykidsu6 + nevermarried + age35p + 1', df).fit()




In [107]:
x = sum_table([reg_rf1, reg_rf, reg_rf2], \
          [ 'First Stage','Reduced Form', 'OLS'], \
         ['treatment','ft15', 'imm', 'hsgrad', 'agelt25', 'working_at_baseline',
       'anykidsu6','nevermarried'],\
          'Table 3 - Reduced form for the probability of being on Welfare')

print(x)

Table 3 - Reduced form for the probability of being on Welfare
                    First Stage Reduced Form    OLS    
-------------------------------------------------------
treatment           0.1422***   -0.1475***             
                    (0.0113)    (0.0122)               
ft15                                         -0.5375***
                                             (0.0134)  
imm                 -0.0556***  0.0405**     0.0083    
                    (0.0170)    (0.0183)     (0.0161)  
hsgrad              0.0924***   -0.1053***   -0.0574***
                    (0.0115)    (0.0124)     (0.0109)  
agelt25             0.0178      -0.0336*     -0.0208   
                    (0.0169)    (0.0182)     (0.0160)  
working_at_baseline 0.2715***   -0.2223***   -0.0755***
                    (0.0146)    (0.0157)     (0.0143)  
anykidsu6           -0.0054     -0.0057      0.0081    
                    (0.0128)    (0.0138)     (0.0132)  
nevermarried        -0.0013     0.0280** 

#### I did not know how to proceed adding the instrumental variable regression into my table. The table below is the iV regression.

In [111]:
print(mod15)

                          IV-2SLS Estimation Summary                          
Dep. Variable:              welfare15   R-squared:                      0.0972
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0957
No. Observations:                4796   F-statistic:                    475.23
Date:                Wed, Mar 13 2019   P-value (F-stat)                0.0000
Time:                        17:44:30   Distribution:                  chi2(8)
Cov. Estimator:                robust                                         
                                                                              
                                  Parameter Estimates                                  
                     Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
---------------------------------------------------------------------------------------
Intercept               0.9463     0.0203     46.666     0.0000      0.9066      0.9860
imm             

Yes there is a slight difference in the probabilities in the coefficient. For example, for the IV model, without the other covariates, ft15 is roughly- 1.037 whereas when the covariates are included ft15 is roughly